In [1]:
import gym

from collections import deque


import numpy as np
import scipy.signal

import tensorflow as tf
from sklearn.utils import shuffle

seed = 0
np.random.seed(seed)

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Policy Function

In [2]:
class Policy(object):
    def __init__(self, obs_dim, act_dim, clip_range=0.2,
                 epochs=10, lr=3e-5, hdim=64, mdn_weight="sparsemax", n_mixture=4, max_std=1.0,
                 seed=0,alpha=1.0):
        self.alpha=alpha
        self.seed=seed
        
        self.obs_dim = obs_dim
        self.act_dim = act_dim
        
        self.clip_range = clip_range
        
        self.epochs = epochs
        self.lr = lr
        self.hdim = hdim
        self.mdn_weight = mdn_weight
        self.n_mixture = n_mixture
        self.max_std = max_std
        
        self._build_graph()
        self._init_session()

    def _build_graph(self):
        self.g = tf.Graph()
        with self.g.as_default():
            self._placeholders()
            self._policy_nn()
            self._logprob()
            self._kl_entropy()
            self._loss_train_op()
            self.init = tf.global_variables_initializer()
            self.variables = tf.global_variables()
            
    def _placeholders(self):
        # observations, actions and advantages:
        self.obs_ph = tf.placeholder(tf.float32, (None, self.obs_dim), 'obs')
        self.act_ph = tf.placeholder(tf.float32, (None, self.act_dim), 'act')
        self.advantages_ph = tf.placeholder(tf.float32, (None,), 'advantages')

        # learning rate:
        self.lr_ph = tf.placeholder(tf.float32, (), 'lr')
        
        # place holder for old parameters
        self.old_std_ph = tf.placeholder(tf.float32, (None, self.act_dim, self.n_mixture), 'old_std')
        self.old_mean_ph = tf.placeholder(tf.float32, (None, self.act_dim, self.n_mixture), 'old_means')
        self.old_pi_ph = tf.placeholder(tf.float32, (None, self.n_mixture), 'old_pi')

    def _policy_nn(self):
        
        hid1_size = self.hdim
        hid2_size = self.hdim
        
        # TWO HIDDEN LAYERS
        out = tf.layers.dense(self.obs_ph, hid1_size, tf.tanh,
                              kernel_initializer=tf.random_normal_initializer(stddev=0.01,seed= self.seed), name="h1")
        out = tf.layers.dense(out, hid2_size, tf.tanh,
                              kernel_initializer=tf.random_normal_initializer(stddev=0.01,seed= self.seed), name="h2")
        means = tf.layers.dense(out, self.act_dim*self.n_mixture,
                                kernel_initializer=tf.random_normal_initializer(stddev=0.01,seed= self.seed), 
                                name="flat_mean")
        self.mean = tf.reshape(means,shape=[-1,self.act_dim,self.n_mixture], name="mean")
        logits_std = tf.layers.dense(out, self.act_dim*self.n_mixture,
                                kernel_initializer=tf.random_normal_initializer(stddev=0.01,seed= self.seed), 
                                name="flat_logits_std")
        self.std = tf.reshape(self.max_std*tf.sigmoid(logits_std),shape=[-1,self.act_dim,self.n_mixture], name="std")
        if self.mdn_weight=="softmax":
            self.pi = tf.nn.softmax(tf.layers.dense(out, self.n_mixture,
                                                kernel_initializer=tf.random_normal_initializer(stddev=0.01,seed= self.seed), name="pi"))
        elif self.mdn_weight=="sparsemax":
            self.pi = tf.contrib.sparsemax.sparsemax(tf.layers.dense(out, self.n_mixture,
                                                kernel_initializer=tf.random_normal_initializer(stddev=0.01,seed= self.seed), name="pi"))
        
    def _logprob(self):
        # PROBABILITY WITH TRAINING PARAMETER
        y = self.act_ph 
        mu = self.mean
        sigma = self.std
        pi = self.pi
        
        quadratics = -0.5*tf.reduce_sum(tf.square((tf.tile(y[:,:,tf.newaxis],[1,1,self.n_mixture])-mu)/sigma),axis=1)
        logdet = -0.5*tf.reduce_sum(tf.log(sigma),axis=1)
        logconstant = - 0.5*self.act_dim*np.log(2.*np.pi)
        logpi = tf.log(pi + 1e-8)
        
        exponents = quadratics + logdet + logconstant + logpi
        logprobs = tf.reduce_logsumexp(exponents,axis=1)
        
        self.logp = logprobs

        old_mu_ph = self.old_mean_ph
        old_sigma_ph = self.old_std_ph
        old_pi_ph = self.old_pi_ph
    
        quadratics = -0.5*tf.reduce_sum(tf.square((tf.tile(y[:,:,tf.newaxis],[1,1,self.n_mixture])-old_mu_ph)/old_sigma_ph),axis=1)
        logdet = -0.5*tf.reduce_sum(tf.log(old_sigma_ph),axis=1)
        logconstant = - 0.5*self.act_dim*np.log(2.*np.pi)
        logpi = tf.log(old_pi_ph + 1e-8)
        
        exponents = quadratics + logdet + logconstant + logpi
        old_logprobs = tf.reduce_logsumexp(exponents,axis=1)
        
        self.logp_old = old_logprobs
        
    def _kl_entropy(self):
        
        def energy(mu1,std1,pi1,mu2,std2,pi2):
            energy_components = []
            for i in range(self.n_mixture):
                for j in range(self.n_mixture):
                    mu1i = mu1[:,:,i] 
                    mu2j = mu2[:,:,j]
                    std1i = std1[:,:,i]
                    std2j = std2[:,:,j]
                    pi1i = pi1[:,i]
                    pi2j = pi2[:,j]
                    energy_components.append(pi1i*pi2j * tf.exp(-0.5*tf.reduce_sum(((mu1i - mu2j)/(std1i+std2j))**2+2.*tf.log(std1i+std2j)+np.log(2*np.pi),axis=1)))
            return tf.reduce_sum(tf.stack(energy_components,axis=1),axis=1) 
            
        mean, std, weight = self.mean, self.std, self.pi
        old_mean, old_std, old_weight = self.old_mean_ph, self.old_std_ph, self.old_pi_ph

#         weight = weight/tf.reduce_sum(weight,axis=1,keep_dims=True)
#         old_weight = old_weight/tf.reduce_sum(old_weight,axis=1,keep_dims=True)
        
        if self.mdn_weight=="softmax":
            self.entropy = tf.reduce_sum(self.pi*(-tf.log(self.pi) + 0.5 * (self.act_dim * (np.log(2 * np.pi) + 1) +
                                                                        tf.reduce_sum(tf.log(std),axis=1))),axis=1)
            self.entropy = tf.reduce_mean(self.entropy)
        elif self.mdn_weight=="sparsemax":
            self.entropy = tf.reduce_mean(0.5*(1-energy(mean, std, weight,mean, std, weight)))
            
        log_det_cov_old = tf.reduce_sum(tf.log(old_std),axis=1)
        log_det_cov_new = tf.reduce_sum(tf.log(std),axis=1)
        tr_old_new = tf.reduce_sum(old_std/std,axis=1)

        kl = tf.reduce_sum(old_weight*tf.log((old_weight+1e-8)/(weight+1e-8)) + 0.5 * old_weight*(log_det_cov_new - log_det_cov_old + tr_old_new +
                         tf.reduce_sum(tf.square((mean - old_mean)/std),axis=1) - self.act_dim),axis=1)
        self.kl = tf.reduce_mean(kl)
        
    def _loss_train_op(self):
        
        # Proximal Policy Optimization CLIPPED LOSS FUNCTION
#         ratio = tf.exp(self.logp - self.logp_old) 
#         clipped_ratio = tf.clip_by_value(ratio,clip_value_min=1-self.clip_range,clip_value_max=1+self.clip_range) 
#         self.loss = -tf.reduce_mean(tf.minimum(self.advantages_ph*ratio,self.advantages_ph*clipped_ratio))
                
        def energy(mu1,std1,pi1,mu2,std2,pi2):
            energy_components = []
            for i in range(self.n_mixture):
                for j in range(self.n_mixture):
                    mu1i = mu1[:,:,i] 
                    mu2j = mu2[:,:,j]
                    std1i = std1[:,:,i]
                    std2j = std2[:,:,j]
                    pi1i = pi1[:,i]
                    pi2j = pi2[:,j]
                    energy_components.append(pi1i*pi2j * tf.exp(-0.5*tf.reduce_sum(((mu1i - mu2j)/(std1i+std2j))**2+2.*tf.log(std1i+std2j)+np.log(2*np.pi),axis=1)))
            return tf.reduce_sum(tf.stack(energy_components,axis=1),axis=1) 
            
        mean, std, weight = self.mean, self.std, self.pi
        
        alpha = self.alpha
        self.error = tf.maximum(self.advantages_ph + alpha*(0.5 + 0.5*energy(mean, std, weight, mean, std, weight)), 
                                0)- alpha*tf.exp(self.logp)
        self.loss = tf.reduce_mean(tf.square(self.error))
        # OPTIMIZER 
        optimizer = tf.train.AdamOptimizer(self.lr_ph)
        self.train_op = optimizer.minimize(self.loss)

    def _init_session(self):
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        self.sess = tf.Session(config=config,graph=self.g)
        self.sess.run(self.init)

    def sample(self, obs): # SAMPLE FROM POLICY
        feed_dict = {self.obs_ph: obs}
        pi, mu, sigma = self.sess.run([self.pi, self.mean, self.std],feed_dict=feed_dict)
        pi = (pi+1e-8)/np.sum(pi+1e-8,axis=1,keepdims=True)
        sigma = sigma
        n_points = np.shape(obs)[0]
        
        _y_sampled = np.zeros([n_points,self.act_dim])
        for i in range(n_points):
            k = np.random.choice(self.n_mixture,p=pi[i,:])
            _y_sampled[i,:] = mu[i,:,k] + np.random.randn(1,self.act_dim)*sigma[i,:,k]
        return _y_sampled
        
    def control(self, obs): # COMPUTE MEAN
        feed_dict = {self.obs_ph: obs}
        pi, mu, sigma = self.sess.run([self.pi, self.mean, self.std],feed_dict=feed_dict)
        pi = (pi+1e-8)/np.sum(pi+1e-8,axis=1,keepdims=True)
        sigma = sigma
        n_points = np.shape(obs)[0]
        
        _y_sampled = np.zeros([n_points,self.act_dim])
        for i in range(n_points):
            k = np.argmax(pi[i,:])
            _y_sampled[i,:] = mu[i,:,k] + np.random.randn(1,self.act_dim)*sigma[i,:,k]
        return _y_sampled        
    
    def update(self, observes, actions, advantages, batch_size = 128): # TRAIN POLICY
        
        num_batches = max(observes.shape[0] // batch_size, 1)
        batch_size = observes.shape[0] // num_batches
        
        old_means_np, old_std_np, old_pi_np = self.sess.run([self.mean, self.std, self.pi],{self.obs_ph: observes}) # COMPUTE OLD PARAMTER
        for e in range(self.epochs):
            observes, actions, advantages, old_means_np, old_std_np = shuffle(observes, actions, advantages, old_means_np, old_std_np, random_state=self.seed)
            for j in range(num_batches): 
                start = j * batch_size
                end = (j + 1) * batch_size
                feed_dict = {self.obs_ph: observes[start:end,:],
                     self.act_ph: actions[start:end,:],
                     self.advantages_ph: advantages[start:end],
                     self.old_std_ph: old_std_np[start:end,:,:],
                     self.old_mean_ph: old_means_np[start:end,:,:],
                     self.old_pi_ph: old_pi_np[start:end,:],
                     self.lr_ph: self.lr}        
                self.sess.run(self.train_op, feed_dict)
            
        feed_dict = {self.obs_ph: observes,
                 self.act_ph: actions,
                 self.advantages_ph: advantages,
                 self.old_std_ph: old_std_np,
                 self.old_mean_ph: old_means_np,
                 self.old_pi_ph: old_pi_np,
                 self.lr_ph: self.lr}             
        loss, kl, entropy = self.sess.run([self.loss, self.kl, self.entropy], feed_dict)
        return loss, kl, entropy
    
    def close_sess(self):
        self.sess.close()

# Value Function

In [3]:
class Value(object):
    def __init__(self, obs_dim, epochs=20, lr=1e-4, hdim=64, seed=0):
        self.seed = seed
    
        self.obs_dim = obs_dim
        self.epochs = epochs
        self.lr = lr
        self.hdim = hdim
        
        self._build_graph()
        self._init_session()
        
    def _build_graph(self):
        self.g = tf.Graph()
        with self.g.as_default():
            self.obs_ph = tf.placeholder(tf.float32, (None, self.obs_dim), 'obs_valfunc')
            self.val_ph = tf.placeholder(tf.float32, (None,), 'val_valfunc')
            
            hid1_size = self.hdim 
            hid2_size = self.hdim 
            
            out = tf.layers.dense(self.obs_ph, hid1_size, tf.tanh,
                                  kernel_initializer=tf.random_normal_initializer(
                                      stddev=0.01,seed=self.seed), name="h1")
            out = tf.layers.dense(out, hid2_size, tf.tanh,
                                  kernel_initializer=tf.random_normal_initializer(
                                      stddev=0.01,seed=self.seed), name="h2")
            out = tf.layers.dense(out, 1,
                                  kernel_initializer=tf.random_normal_initializer(
                                      stddev=0.01,seed=self.seed), name='output')
            self.out = tf.squeeze(out)
            
            # L2 LOSS
            self.loss = tf.reduce_mean(tf.square(self.out - self.val_ph))
            
            # OPTIMIZER
            optimizer = tf.train.AdamOptimizer(self.lr)
            self.train_op = optimizer.minimize(self.loss)
            
            self.init = tf.global_variables_initializer()
            self.variables = tf.global_variables()
    
    def _init_session(self):
        config = tf.ConfigProto()
        config.gpu_options.allow_growth = True
        self.sess = tf.Session(config=config,graph=self.g)
        self.sess.run(self.init)

    def fit(self, x, y, batch_size=32):
        num_batches = max(x.shape[0] // batch_size, 1)
        x_train, y_train = x, y
        for e in range(self.epochs):
            x_train, y_train = shuffle(x_train, y_train, random_state=self.seed)
            for j in range(num_batches):
                start = j * batch_size
                end = (j + 1) * batch_size
                feed_dict = {self.obs_ph: x_train[start:end, :],
                             self.val_ph: y_train[start:end]}
                self.sess.run([self.train_op], feed_dict=feed_dict)
        feed_dict = {self.obs_ph: x_train,
                     self.val_ph: y_train}
        loss, = self.sess.run([self.loss], feed_dict=feed_dict)
        return loss

    def predict(self, x): # PREDICT VALUE OF THE GIVEN STATE
        feed_dict = {self.obs_ph: x}
        y_hat = self.sess.run(self.out, feed_dict=feed_dict)
        return np.squeeze(y_hat)

    def close_sess(self):
        self.sess.close()

# Helper Functions

In [4]:
def discount(x, gamma=0.99): # compute discount
    return scipy.signal.lfilter([1.0], [1.0, -gamma], x[::-1])[::-1]

def add_value(trajectories, val_func): # Add value estimation for each trajectories
    for trajectory in trajectories:
        observes = trajectory['observes']
        values = val_func.predict(observes)
        trajectory['values'] = values

def add_gae(trajectories, gamma=0.99, lam=0.98): # generalized advantage estimation (for training stability)
    for trajectory in trajectories:
        rewards = trajectory['rewards']
        values = trajectory['values']
        
        # temporal differences
        tds = rewards - values + np.append(values[1:] * gamma, 0)
        advantages = discount(tds, gamma * lam)
        
        trajectory['advantages'] = advantages
        trajectory['returns'] = values+advantages

def build_train_set(trajectories):
    observes = np.concatenate([t['observes'] for t in trajectories])
    actions = np.concatenate([t['actions'] for t in trajectories])
    returns = np.concatenate([t['returns'] for t in trajectories])
    advantages = np.concatenate([t['advantages'] for t in trajectories])

    # Normalization of advantages 
    # In baselines, which is a github repo including implementation of PPO by OpenAI, 
    # all policy gradient methods use advantage normalization trick as belows.
    # The insight under this trick is that it tries to move policy parameter towards locally maximum point.
    # Sometimes, this trick doesnot work.
    advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-6)

    return observes, actions, advantages, returns

def run_episode(env, policy, animate=False, evaluation=False): # Run policy and collect (state, action, reward) pairs
    obs = env.reset()
    observes, actions, rewards, infos = [], [], [], []
    done = False
    while not done:
        if animate:
            env.render()
        obs = obs.astype(np.float32).reshape((1, -1))
        observes.append(obs)
        if evaluation:
            action = policy.control(obs).reshape((1, -1)).astype(np.float32)
        else:
            action = policy.sample(obs).reshape((1, -1)).astype(np.float32)
        actions.append(action)
        obs, reward, done, info = env.step(action)
        if not isinstance(reward, float):
            reward = np.asscalar(reward)
        rewards.append(reward)
        infos.append(info)
        
    return (np.concatenate(observes), np.concatenate(actions), np.array(rewards, dtype=np.float32), infos)

def run_policy(env, policy, episodes, evaluation=False): # collect trajectories. if 'evaluation' is ture, then only mean value of policy distribution is used without sampling.
    total_steps = 0
    trajectories = []
    for e in range(episodes):
        observes, actions, rewards, infos = run_episode(env, policy, evaluation=evaluation)
        total_steps += observes.shape[0]
        trajectory = {'observes': observes,
                      'actions': actions,
                      'rewards': rewards,
                      'infos': infos}
        trajectories.append(trajectory)
    return trajectories

# Train

In [5]:
env = gym.make('Pendulum-v0')

env.seed(seed=seed)

obs_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]

policy = Policy(obs_dim, act_dim, epochs=30, hdim=32, lr=3e-4, clip_range=0.2,seed=seed)
val_func = Value(obs_dim, epochs=50, hdim=32, lr=1e-3, seed=seed)

episode_size = 100
batch_size = 64
nupdates = 100

for update in range(nupdates+1):

    trajectories = run_policy(env, policy, episodes=episode_size)

    add_value(trajectories, val_func)
    add_gae(trajectories)
    observes, actions, advantages, returns = build_train_set(trajectories)

    pol_loss, pol_kl, pol_entropy = policy.update(observes, actions, advantages, batch_size=batch_size)  
    vf_loss = val_func.fit(observes, returns,batch_size=batch_size)
    
    mean_ret = np.mean([np.sum(t['rewards']) for t in trajectories])
    if (update%5) == 0:
        print('[{}/{}] Mean Ret : {:.3f}, Value Loss : {:.3f}, Policy loss : {:.5f}, Policy KL : {:.5f}, Policy Entropy : {:.3f} ***'.format(
                            update, nupdates, mean_ret, vf_loss, pol_loss, pol_kl, pol_entropy))

[2018-06-06 16:11:56,968] Making new env: Pendulum-v0


[0/100] Mean Ret : -1225.243, Value Loss : 2520.135, Policy loss : 0.91378, Policy KL : 38181.12500, Policy Entropy : 0.229 ***
[5/100] Mean Ret : -1003.503, Value Loss : 5033.092, Policy loss : 1.11494, Policy KL : 385161493479424.00000, Policy Entropy : 0.402 ***
[10/100] Mean Ret : -903.130, Value Loss : 4928.692, Policy loss : 1.11335, Policy KL : 26140194816.00000, Policy Entropy : 0.400 ***
[15/100] Mean Ret : -854.486, Value Loss : 4656.898, Policy loss : 1.11338, Policy KL : 82307383296.00000, Policy Entropy : 0.400 ***
[20/100] Mean Ret : -783.526, Value Loss : 4081.543, Policy loss : 1.11193, Policy KL : 8950589440.00000, Policy Entropy : 0.400 ***
[25/100] Mean Ret : -305.748, Value Loss : 1130.750, Policy loss : 1.09559, Policy KL : 664681408.00000, Policy Entropy : 0.401 ***
[30/100] Mean Ret : -221.270, Value Loss : 257.654, Policy loss : 0.49526, Policy KL : 283.76273, Policy Entropy : 0.264 ***
[35/100] Mean Ret : -149.261, Value Loss : 3.299, Policy loss : 1.10694, Pol

KeyboardInterrupt: 

In [ ]:
trajectories = run_policy(env, policy, episodes=100, evaluation=True)
mean_ret = np.mean([np.sum(t['rewards']) for t in trajectories])
print('Results: {}'.format(mean_ret))